# Stage 5, Report

https://github.com/anhaidgroup/py_entitymatching/blob/master/notebooks/vldb_demo/Demo_notebook_v6.ipynb

In [2]:
import py_entitymatching as em
import os
import pandas as pd

# specify filepaths for tables A and B. 
path_A = 'tableA.csv'
path_B = 'tableB.csv'
# read table A; table A has 'ID' as the key attribute
A = em.read_csv_metadata(path_A, key='id')
# read table B; table B has 'ID' as the key attribute
B = em.read_csv_metadata(path_B, key='id')

# Filling in Missing Values

In [3]:
# Impute missing values

# Manually set metadata properties, as current py_entitymatching.impute_table()
# requires 'fk_ltable', 'fk_rtable', 'ltable', 'rtable' properties
em.set_property(A, 'fk_ltable', 'id')
em.set_property(A, 'fk_rtable', 'id')
em.set_property(A, 'ltable', A)
em.set_property(A, 'rtable', A)

A_all_attrs = list(A.columns.values)
A_impute_attrs = ['year','min_num_players','max_num_players','min_gameplay_time','max_gameplay_time','min_age']
A_exclude_attrs = list(set(A_all_attrs) - set(A_impute_attrs))
A1 = em.impute_table(A, exclude_attrs=A_exclude_attrs, missing_val='NaN', strategy='most_frequent', axis=0, val_all_nans=0, verbose=True)

# Compare number of missing values to check the results
print(sum(A['min_num_players'].isnull()))
print(sum(A1['min_num_players'].isnull()))

# Do the same thing for B
em.set_property(B, 'fk_ltable', 'id')
em.set_property(B, 'fk_rtable', 'id')
em.set_property(B, 'ltable', B)
em.set_property(B, 'rtable', B)

B_all_attrs = list(B.columns.values)
# TODO: add 'min_age'
B_impute_attrs = ['year','min_num_players','max_num_players','min_gameplay_time','max_gameplay_time']
B_exclude_attrs = list(set(B_all_attrs) - set(B_impute_attrs))
B1 = em.impute_table(B, exclude_attrs=B_exclude_attrs, missing_val='NaN', strategy='most_frequent', axis=0, val_all_nans=0, verbose=True)

# Compare number of missing values to check the results
print(sum(B['min_num_players'].isnull()))
print(sum(B1['min_num_players'].isnull()))


65
0
5244
0


In [6]:
# Load the pre-labeled data
S = em.read_csv_metadata('sample_labeled.csv', 
                         key='_id',
                         ltable=A1, rtable=B1, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')

path_total_cand_set = 'candidate_set_C1.csv'
total_cand_set = em.read_csv_metadata(path_total_cand_set, 
                         key='_id',
                         ltable=A1, rtable=B1, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')

In [7]:
# Split S into I an J
IJ = em.split_train_test(S, train_proportion=0.75, random_state=35)
I = IJ['train']
J = IJ['test']

In [8]:
corres = em.get_attr_corres(A1, B1)

{'rtable':           id  \
0          1   
1          2   
2          3   
3          4   
4          5   
5          6   
6          7   
7          8   
8          9   
9         10   
10        11   
11        12   
12        13   
13        14   
14        15   
15        16   
16        17   
17        18   
18        19   
19        20   
20        21   
21        22   
22        23   
23        24   
24        25   
25        26   
26        27   
27        28   
28        29   
29        30   
...      ...   
10266  10269   
10267  10270   
10268  10271   
10269  10272   
10270  10273   
10271  10274   
10272  10275   
10273  10276   
10274  10277   
10275  10278   
10276  10279   
10277  10280   
10278  10281   
10279  10282   
10280  10283   
10281  10284   
10282  10285   
10283  10286   
10284  10287   
10285  10288   
10286  10289   
10287  10290   
10288  10291   
10289  10292   
10290  10293   
10291  10294   
10292  10295   
10293  10296   
10294  10297   
10295  10298 

# Generating Features

Here, we generate all the features we decided upon after our final iteration of cross validation and debugging. We only use the relevant subset of all these features in the reported iterations below.

In [9]:
# Generate a set of features
#import pdb; pdb.set_trace();
import py_entitymatching.feature.attributeutils as au
import py_entitymatching.feature.simfunctions as sim
import py_entitymatching.feature.tokenizers as tok

ltable = A1
rtable = B1

# Get similarity functions for generating the features for matching
sim_funcs = sim.get_sim_funs_for_matching()
# Get tokenizer functions for generating the features for matching
tok_funcs = tok.get_tokenizers_for_matching()

# Get the attribute types of the input tables
attr_types_ltable = au.get_attr_types(ltable)
attr_types_rtable = au.get_attr_types(rtable)

# Get the attribute correspondence between the input tables
attr_corres = au.get_attr_corres(ltable, rtable)
print(attr_types_ltable['name'])
print(attr_types_rtable['name'])
attr_types_ltable['name'] = 'str_bt_5w_10w'
attr_types_rtable['name'] = 'str_bt_5w_10w'



# Get the features
F = em.get_features(ltable, rtable, attr_types_ltable,
                                 attr_types_rtable, attr_corres,
                                 tok_funcs, sim_funcs)

#F = em.get_features_for_matching(A1, B1)
print(F['feature_name'])

# Convert the I into a set of feature vectors using F
# Here, we add name edit distance as a feature
include_features_2 = [
    'min_num_players_min_num_players_lev_dist',
    'max_num_players_max_num_players_lev_dist',
    'min_gameplay_time_min_gameplay_time_lev_dist',
    'max_gameplay_time_max_gameplay_time_lev_dist',
    'name_name_lev_dist'
]
F_2 = F.loc[F['feature_name'].isin(include_features_2)]

str_bt_1w_5w
str_bt_5w_10w
0                                           id_id_exm
1                                           id_id_anm
2                                      id_id_lev_dist
3                                       id_id_lev_sim
4                           name_name_jac_qgm_3_qgm_3
5                       name_name_cos_dlm_dc0_dlm_dc0
6                                       name_name_mel
7                                  name_name_lev_dist
8                                   name_name_lev_sim
9                                       year_year_exm
10                                      year_year_anm
11                                 year_year_lev_dist
12                                  year_year_lev_sim
13                   num_players_num_players_lev_dist
14                    num_players_num_players_lev_sim
15                        num_players_num_players_jar
16                        num_players_num_players_jwn
17                        num_players_num_players_exm
1

# Generate training set

In [11]:
# Apply train, test set evaluation
I_table = em.extract_feature_vecs(I, feature_table=F_2, attrs_after='label', show_progress=False)
J_table = em.extract_feature_vecs(J, feature_table=F_2, attrs_after='label', show_progress=False)

In [52]:
total_cand_set_features = em.extract_feature_vecs(total_cand_set, feature_table=F_2, show_progress=False)

m = em.LogRegMatcher(name='LogReg', random_state=0)

m.fit(table=I_table, exclude_attrs=['_id', 'ltable_id', 'rtable_id','label'], target_attr='label')

total_cand_set_features['prediction'] = m.predict(
    table=total_cand_set_features, 
    exclude_attrs=['_id', 'ltable_id', 'rtable_id'],
)

In [55]:
# Join tables on matched tuples
match_tuples = total_cand_set_features[total_cand_set_features['prediction']==1]
match_tuples = match_tuples[['ltable_id','rtable_id']]
A1['ltable_id'] = A1['id']
B1['rtable_id'] = B1['id']
joined_tables = pd.merge(match_tuples, A1, how='left', on='ltable_id')
joined_tables = pd.merge(joined_tables, B1, how='left', on='rtable_id')


In [56]:
for n in A1.columns: 
    if not n in ['_id', 'ltable_id', 'rtable_id']:
        joined_tables[n] =  joined_tables.apply((lambda row: row[n+'_y'] if pd.isnull(row[n+'_x']) else row[n+'_x']), axis=1)
        joined_tables = joined_tables.drop(n+'_x', axis=1).drop(n+'_y',axis=1)

joined_tables.to_csv('joined_table.csv')

In [57]:
joined_tables

,ltable_id,rtable_id,id,name,year,rating,rank,num_players,min_num_players,max_num_players,...,complexity_weight,category,mechanisms,type,BGG_link,store_names,store_prices,links_to_buy,availability,international_store
0,244,5464,244,Win Lose or Draw,1988.0,5.2,12089.0,3-8,3.0,8.0,...,1.00,Movies / TV / Radio theme;Party Game,Paper-and-Pencil;Partnerships;Roll / Spin and Move,NaN,http://www.boardgamegeek.com/boardgame/6800,"Noble Knight Games,Noble Knight Games,Amazon.com,","$5.00,$9.50,$24.95,Prices from $ 4.00",http://www.nobleknight.com/ProductDetailSearch.asp_Q_ProductID_E_2147467594_A_InventoryID_E_2147...,"In Stock,In Stock,In Stock,In Stock","False,False,False,False"
1,1960,541,1960,Vanity Chase,1988.0,6.2,NaN,2-4,2.0,4.0,...,NaN,Deduction;Humor;Puzzle;Racing;Word Game,Pattern Recognition;Roll / Spin and Move,NaN,http://www.boardgamegeek.com/boardgame/8431,"Noble Knight Games,Amazon.com,","$15.00,$52.96,Prices from $ 12.00",http://www.nobleknight.com/ProductDetailSearch.asp_Q_ProductID_E_2147578406_A_InventoryID_E_2148...,"In Stock,In Stock,In Stock","False,False,False"
2,2295,3685,2295,Grand Army of the Republic,1988.0,5.8,9874.0,2-4,2.0,4.0,...,2.88,American Civil War;Wargame,Area Movement;Simulation,NaN,http://www.boardgamegeek.com/boardgame/8753,"Noble Knight Games,Noble Knight Games,Amazon.com,","$10.00,$14.95,$32.97,Prices from $ 43.00",http://www.nobleknight.com/ProductDetailSearch.asp_Q_ProductID_E_7585_A_InventoryID_E_2147567022...,"In Stock,In Stock,In Stock,In Stock","False,False,False,False"
3,5971,1462,5971,Buck Rogers: Battle for the 25th Century Game,1988.0,6.4,2489.0,2-6,2.0,6.0,...,2.67,Comic Book / Strip;Movies / TV / Radio theme;Science Fiction;Wargame,NaN,Characters: Buck Rogers,http://www.boardgamegeek.com/boardgame/548,"Noble Knight Games,Noble Knight Games,Amazon.com,","$65.00,$97.95,$174.99,Prices from $ 55.00",http://www.nobleknight.com/ProductDetailSearch.asp_Q_ProductID_E_16822_A_InventoryID_E_214840163...,"In Stock,In Stock,In Stock,In Stock","False,False,False,False"
4,611,1917,611,Legal Decision,1985.0,2.8,NaN,3-6,3.0,6.0,...,3.00,Educational,Partnerships;Roll / Spin and Move,NaN,http://www.boardgamegeek.com/boardgame/7146,"Amazon.com,BoardGameCo,","$9.50,$12.99,Prices from $ 19.00",http://www.amazon.com/Legal-Decisions-Limited-First-Edition/dp/B009QRYYB4%3FSubscriptionId%3DAKI...,"In Stock,In Stock,In Stock","False,False,False"
5,2878,3832,2878,East Wind Rain,1985.0,4.8,11478.0,2,2.0,2.0,...,3.67,Wargame;World War II,Secret Unit Deployment,NaN,http://www.boardgamegeek.com/boardgame/9357,"Amazon.com,Noble Knight Games,","-,$44.00,Prices from $ 5.00",http://www.amazon.com/WWW-Pacific-1941-45-Board-Edtion/dp/B007CDOCDY%3FSubscriptionId%3DAKIAJ4BC...,"In Stock,In Stock,In Stock","False,False,False"
6,3108,5803,3108,All My Children,1985.0,4.6,NaN,2-6,2.0,6.0,...,1.00,Bluffing;Card Game;Humor;Movies / TV / Radio theme,Hand Management;Point to Point Movement;Press Your Luck;Role Playing;Roll / Spin and Move;Trick-...,NaN,http://www.boardgamegeek.com/boardgame/9575,"Noble Knight Games,Amazon.com,","$15.00,$32.99,Prices from $ 8.00",http://www.nobleknight.com/ProductDetailSearch.asp_Q_ProductID_E_51_A_InventoryID_E_2148193209_A...,"In Stock,In Stock,In Stock","False,False,False"
7,3258,8937,3258,Main Battle Area,1985.0,5.5,NaN,2,2.0,2.0,...,3.00,Modern Warfare;Wargame,Hex-and-Counter,NaN,http://www.boardgamegeek.com/boardgame/9720,"Amazon.com,Noble Knight Games,Noble Knight Games,","$21.80,$35.00,$38.00,Prices from $ 14.00",http://www.amazon.com/Animal-Crossing-New-Leaf-Nintendo-3DS/dp/B0053BCML6%3Fpsc%3D1%26Subscripti...,"In Stock,In Stock,In Stock,In Stock","False,False,False,False"
8,3742,4896,3742,Air Cav,1985.0,5.9,7699.0,2,2.0,2.0,...,2.83,Aviation / Flight;Modern Warfare;Wargame,Hex-and-Counter,NaN,http://www.boardgamegeek.com/boardgame/3309,"Amazon.com,Noble Knight Games,Noble Knight Games,","-,$8.00,$15.00,Prices from $ 3.00",http://www.amazon.com/Helicopter-Warfare-Eighties-Bookshelf-Boardg

# Adventure Time!

In [58]:
import matplotlib as plt

In [59]:
joined_tables.plot(x='year', y='rating')

/Users/Patron/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [60]:
plt.show()

AttributeError: 'module' object has no attribute 'show'

TypeError: 'module' object is not callable